<a href="https://colab.research.google.com/github/pecha2k2/Mumble-Channel-Viewer/blob/master/Copiar_rclone_googlecollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

base_dir = '~/sync_task'
cache_dir = os.path.join(base_dir, 'cache')
tmp_dir = os.path.join(base_dir, 'tmp')

# Ensure directories exist
for dir_path in [base_dir, cache_dir, tmp_dir]:
    os.makedirs(os.path.expanduser(dir_path), exist_ok=True)

print("Directories are set up.")

Directories are set up.


In [ ]:
!curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4707  100  4707    0     0   4059      0  0:00:01  0:00:01 --:--:--  4061

The latest version of rclone rclone v1.63.1 is already installed.



In [ ]:
from google.colab import files

# Upload the rclone config file
uploaded = files.upload()

# Check if the correct file was uploaded and move it to the sync_task directory
for fn in uploaded.keys():
    if "rclone.conf" in fn:
        print('User uploaded rclone config file.')
        !mv rclone.conf ~/sync_task/rclone.conf
        print("rclone configuration moved to ~/sync_task/")
    else:
        print(f'Unexpected file "{fn}". Please upload rclone.conf.')

Saving rclone.conf to rclone.conf
User uploaded rclone config file.
rclone configuration moved to ~/sync_task/


In [ ]:
!rclone lsd teamdiario: --config ~/sync_task/rclone.conf && rclone lsd DropBox:/Peliculas --config ~/sync_task/rclone.conf

          -1 2023-07-17 07:49:38        -1 ###ESTRENOS CINE
          -1 2023-07-17 07:50:12        -1 ##SERIES EN EMISION
          -1 2023-07-17 07:50:20        -1 ##SERIES EN EMISION 4K
          -1 2023-07-17 07:50:03        -1 #NOVEDADES MES ACTUAL
          -1 2023-07-31 21:55:51        -1 JULIO 2023
          -1 2000-01-01 00:00:00        -1 4K-MicroUHD
          -1 2000-01-01 00:00:00        -1 4K-UHDremux
          -1 2000-01-01 00:00:00        -1 Conciertos
          -1 2000-01-01 00:00:00        -1 Documental


In [ ]:
import subprocess

# Define rclone command with logging
command = "rclone copy 'teamdiario:/#NOVEDADES MES ACTUAL' DropBox:/mensual/ --config ~/sync_task/rclone.conf -v --log-file ~/sync_task/rclone.log --transfers 15 --checkers 64 --fast-list --cache-dir ~/sync_task/cache --temp-dir ~/sync_task/tmp"
rclone_process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
import time
from IPython.display import clear_output

# Define monitoring period and duration
log_check_interval = 10  # seconds
keep_alive_message_interval = 60  # seconds

last_message_time = 0

while True:
    # Check if the rclone process is still running
    if rclone_process.poll() is not None:
        print("rclone process has finished.")
        break

    clear_output(wait=True)  # Clear the previous output

    # Print last 30 lines of the log file
    log_content = !tail -n 30 ~/sync_task/rclone.log
    for line in log_content:
        print(line)

    # Print a keep-alive message
    current_time = time.time()
    if current_time - last_message_time > keep_alive_message_interval:
        print("Still copying...")
        last_message_time = current_time

    # Sleep before checking the log again
    time.sleep(log_check_interval)

Checks:               578 / 578, 100%
Transferred:            3 / 4, 75%
Elapsed time:    2h42m1.7s
Transferring:
 * 02. Películas 4K (NO N…4K UHDRemux 2160p].mkv: 98% /148.746Gi, 12.659Mi/s, 2m55s

2023/08/13 12:39:48 INFO  : 
Transferred:   	  173.068 GiB / 174.508 GiB, 99%, 12.371 MiB/s, ETA 1m59s
Checks:               578 / 578, 100%
Transferred:            3 / 4, 75%
Elapsed time:    2h43m1.7s
Transferring:
 * 02. Películas 4K (NO N…4K UHDRemux 2160p].mkv: 99% /148.746Gi, 12.371Mi/s, 1m59s

2023/08/13 12:40:48 INFO  : 
Transferred:   	  173.780 GiB / 174.508 GiB, 100%, 12.602 MiB/s, ETA 59s
Checks:               578 / 578, 100%
Transferred:            3 / 4, 75%
Elapsed time:    2h44m1.7s
Transferring:
 * 02. Películas 4K (NO N…4K UHDRemux 2160p].mkv: 99% /148.746Gi, 12.701Mi/s, 58s

2023/08/13 12:41:48 INFO  : 
Transferred:   	  174.508 GiB / 174.508 GiB, 100%, 11.532 MiB/s, ETA 0s
Checks:               578 / 578, 100%
Transferred:            3 / 4, 75%
Elapsed time:    2h45m1.7s

In [ ]:
# Check if the process is still running
if rclone_process.poll() is None:
    rclone_process.terminate()
    print("rclone process terminated.")
else:
    print("rclone process is not running.")

rclone process is not running.
